In [1]:
import cv2
import depthai as dai
import numpy as np

# Q2

## Image stitching function

### Stitches based on SIFT or ORB features

Usage : 

    stitch_2_imgs(img1, img2, feature)
    
    for stitching using ORB features, use "ORB"
    for stitching using SIFT features, use "SIFT"
    

In [2]:
def stitch_2_imgs(img1, img2, feature) : 
    img1_bw = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_bw = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    if feature == "SIFT":
        feat = cv2.SIFT_create() 
    elif feature == "ORB":
        feat = cv2.ORB_create()
    else :
        print("PLease enter correct feature value.")
        return 

    kp_img1, desc_img1 = feat.detectAndCompute(img1_bw, None) 
    kp_img2, desc_img2 = feat.detectAndCompute(img2_bw, None) 

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc_img2, desc_img1, k=2)

    good_points = []
    for m, n in matches: 
        if(m.distance < 0.6*n.distance): 
            good_points.append(m) 

    query_pts = np.float32([kp_img2[m.queryIdx] 
                    .pt for m in good_points]).reshape(-1, 1, 2) 

    train_pts = np.float32([kp_img1[m.trainIdx] 
                    .pt for m in good_points]).reshape(-1, 1, 2) 

    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 

    #wraped image
    dst = cv2.warpPerspective(img2, matrix, ((img1.shape[1] + img2.shape[1]), img2.shape[0])) 

    # now paste them together
    dst[0:img1.shape[0], 0:img1.shape[1]] = img1

    return dst

## Science centre

In [3]:
img1 = cv2.imread('image_stitching/sciencecenter1.jpg')
img2 = cv2.imread('image_stitching/sciencecenter2.jpg')
img3 = cv2.imread('image_stitching/sciencecenter3.jpg')

In [4]:
img2_3 = stitch_2_imgs(img2, img3, "SIFT")
panorm = stitch_2_imgs(img1, img2_3, "SIFT")

cv2.imwrite(f"image_stitching/sciencecent_stiched.jpg", panorm)

True

## Rialto team

In [5]:
img1 = cv2.imread('image_stitching/rialto_team6-1.jpeg')
img2 = cv2.imread('image_stitching/rialto_team6-2.jpeg')
img3 = cv2.imread('image_stitching/rialto_team6-3.jpeg')

In [6]:
img2_3 = stitch_2_imgs(img2, img3, "SIFT")
panorm = stitch_2_imgs(img1, img2_3, "SIFT")

cv2.imwrite(f"image_stitching/rialto_team6_stiched.jpg", panorm)

True

## Sports arena

In [7]:
img1 = cv2.imread('image_stitching/sportsarena1.jpg')
img2 = cv2.imread('image_stitching/sportsarena2.jpg')
img3 = cv2.imread('image_stitching/sportsarena3.jpg')

In [8]:
img2_3 = stitch_2_imgs(img2, img3, "SIFT")
panorm = stitch_2_imgs(img1, img2_3, "SIFT")

cv2.imwrite(f"image_stitching/sportsarena_stiched.jpg", panorm)

True

# Q4

In [19]:
img1 = cv2.imread('video_frames/16679490776731.png')
img2 = cv2.imread('video_frames/16679490737739.png')

In [20]:
img1_2 = stitch_2_imgs(img1, img2, "SIFT")

cv2.imwrite(f"q4_stiched_frames.png", img1_2)

True

# Q5

In [21]:
img1_2 = stitch_2_imgs(img1, img2, "ORB")

cv2.imwrite(f"q5_stiched_frames.png", img1_2)

True